In [24]:
import requests
import re
import json

In [ ]:
def get_raw_data(state, columns):
    base_url = 'https://api.census.gov/data/2018/acs/acs5/profile/'
    parameters = {
        'for': 'county:*',
        'in': f'state:{state}',
        'get': ','.join(['NAME'] + columns)
    }
    return requests.get(base_url, params=parameters)

In [53]:
def clean_data(data):
    return json.loads(data)

In [54]:
r = get_raw_data(
        state='39',
        columns=[f'DP03_00{i}PE' for i in range(52, 62)]
    )

data = clean_data(r.text)

In [60]:
import json
l = json.loads(r.text)